<a href="https://www.kaggle.com/code/kangnurrohman/polarity-determination-3-algorithm-nb-rf-svm?scriptVersionId=113339813" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Install library

In [1]:
!pip install sastrawi
!pip install contractions

import warnings
warnings.filterwarnings('ignore')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 927.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 553.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 2.1 MB/s eta 0:00:00


## Load Library

In [2]:
import re
import tqdm
import nltk
import string
import unicodedata
import numpy as np
import pandas as pd
import contractions

from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

## Load Data

In [3]:
df = pd.read_excel('/kaggle/input/review-of-the-application-pln-mobile/review of the application PLN mobile.xlsx')
df.rename(columns = {'content':'review', 'score':'sentiment'}, inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201042 entries, 0 to 201041
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              201042 non-null  object        
 1   userName              201042 non-null  object        
 2   userImage             201042 non-null  object        
 3   review                201042 non-null  object        
 4   sentiment             201042 non-null  int64         
 5   thumbsUpCount         201042 non-null  int64         
 6   reviewCreatedVersion  174363 non-null  object        
 7   at                    201042 non-null  datetime64[ns]
 8   replyContent          195639 non-null  object        
 9   repliedAt             195639 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 15.3+ MB


In [4]:
df = df.sort_values(by='at', ascending=False)
df = df[['review', 'sentiment']]
df.head()

,review,sentiment
0,Sangat bermanfaat👍👍,5
1,PLN terbaik,5
2,😀,5
3,Mudah dan simpel untuk menikmati pelayanan PLN...,5
4,Baik,5


## Exploratory Data Analysis

In [5]:
df = df.replace({'sentiment' : {1:'negative', 2:'negative', 3:'neutral', 4:'positive', 5: 'positive' }})
df.head()

,review,sentiment
0,Sangat bermanfaat👍👍,positive
1,PLN terbaik,positive
2,😀,positive
3,Mudah dan simpel untuk menikmati pelayanan PLN...,positive
4,Baik,positive


In [6]:
df.sentiment.value_counts()

positive    175750
negative     19925
neutral       5367
Name: sentiment, dtype: int64

In [7]:
#removing neutral
df = df[df.sentiment != "neutral"]

## Data Preprocessing

In [8]:
def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def stopwords_removal(words):
    list_stopwords = nltk.corpus.stopwords.words('indonesian')
    #list_stopwords = stopwords.words('indonesian')
    #list_stopwords.extend([])
    #txt_stopword = pd.read_csv("#", names= ["stopwords"], header = None)
    #list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))
    return [word for word in words if word not in list_stopwords]

def stemmed_wrapper(term):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    return stemmer.stem(term)

In [9]:
def pre_process_corpus(docs):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = strip_html_tags(doc)
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    doc = doc.lower()
    doc = remove_accented_chars(doc)
    doc = contractions.fix(doc)
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()
    doc = word_tokenize(doc)
    doc = stopwords_removal(doc)
    #doc = stemmed_wrapper(doc)
    norm_docs.append(doc)

  norm_docs = [" ".join(word) for word in norm_docs]
  return norm_docs

In [10]:
%%time
df.review = pre_process_corpus(df.review)

100%|██████████| 195675/195675 [01:59<00:00, 1632.29it/s]


CPU times: user 1min 56s, sys: 3.9 s, total: 2min
Wall time: 2min


## TF-IDF

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
text_tf = tf.fit_transform(df['review'].astype('U'))
text_tf

<195675x51127 sparse matrix of type '<class 'numpy.float64'>'
	with 827045 stored elements in Compressed Sparse Row format>

## Splitting Data

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(text_tf, df['sentiment'], test_size=0.1, random_state = 42)

## Handle Data Imbalance (Oversampling)

In [13]:
#OVERSAMPLING

print("Before OverSampling, counts of label 'Positive': {}".format(sum(y_train == 'positive')))
print("Before OverSampling, counts of label 'Negative': {}".format(sum(y_train == 'negative')))

# import SMOTE module from imblearn library
# pip install imblearn (if you don't have imblearn in your system)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label 'positive': {}".format(sum(y_train_res == 'positive')))
print("After OverSampling, counts of label 'negative': {}".format(sum(y_train_res == 'negative')))

Before OverSampling, counts of label 'Positive': 158156
Before OverSampling, counts of label 'Negative': 17951
After OverSampling, the shape of train_X: (316312, 51127)
After OverSampling, the shape of train_y: (316312,) 

After OverSampling, counts of label 'positive': 158156
After OverSampling, counts of label 'negative': 158156


## Naive Bayes

In [14]:
from sklearn.naive_bayes import MultinomialNB  
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import confusion_matrix

clf = MultinomialNB().fit(X_train_res, y_train_res)
predicted = clf.predict(X_test)
print("MultinomialNB Accuracy:" , accuracy_score(y_test,predicted))
print("MultinomialNB Precision:" , precision_score(y_test,predicted, average="macro", pos_label="negative" ))
print("MultinomialNB Recall:" , recall_score(y_test,predicted, average="macro", pos_label="negative"))
print("MultinomialNB f1_score:" , f1_score(y_test,predicted, average="macro", pos_label="negative"))

print(f'confusion matrix:\n {confusion_matrix(y_test, predicted)}')
print('================================================\n') 
print(classification_report(y_test, predicted, zero_division=0))                  

MultinomialNB Accuracy: 0.8823589533932952
MultinomialNB Precision: 0.7243987799834574
MultinomialNB Recall: 0.8999493702317924
MultinomialNB f1_score: 0.7716192357903224
confusion matrix:
 [[ 1820   154]
 [ 2148 15446]]

              precision    recall  f1-score   support

    negative       0.46      0.92      0.61      1974
    positive       0.99      0.88      0.93     17594

    accuracy                           0.88     19568
   macro avg       0.72      0.90      0.77     19568
weighted avg       0.94      0.88      0.90     19568



## SVM

In [15]:
from sklearn import svm

clf = svm.SVC(kernel='linear').fit(X_train_res, y_train_res)
predicted = clf.predict(X_test)
print("Support Vector Machine Accuracy:" , accuracy_score(y_test,predicted))
print("Support Vector Machine Precision:" , precision_score(y_test,predicted, average="macro", pos_label="negative" ))
print("Support Vector Machine Recall:" , recall_score(y_test,predicted, average="macro", pos_label="negative"))
print("Support Vector Machine f1_score:" , f1_score(y_test,predicted, average="macro", pos_label="negative"))

print(f'confusion matrix:\n {confusion_matrix(y_test, predicted)}')
print('================================================\n') 
print(classification_report(y_test, predicted, zero_division=0))  

Support Vector Machine Accuracy: 0.8794971381847915
Support Vector Machine Precision: 0.7199731182795699
Support Vector Machine Recall: 0.8920614458346132
Support Vector Machine f1_score: 0.7660634917435188
confusion matrix:
 [[ 1792   182]
 [ 2176 15418]]

              precision    recall  f1-score   support

    negative       0.45      0.91      0.60      1974
    positive       0.99      0.88      0.93     17594

    accuracy                           0.88     19568
   macro avg       0.72      0.89      0.77     19568
weighted avg       0.93      0.88      0.90     19568



## Random Forest

In [16]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier().fit(X_train_res, y_train_res)
predicted = clf.predict(X_test)
print("Random Forest Accuracy:" , accuracy_score(y_test,predicted))
print("Random Forest Precision:" , precision_score(y_test,predicted, average="macro", pos_label="negative" ))
print("Random Forest Recall:" , recall_score(y_test,predicted, average="macro", pos_label="negative"))
print("Random Forest f1_score:" , f1_score(y_test,predicted, average="macro", pos_label="negative"))

print(f'confusion matrix:\n {confusion_matrix(y_test, predicted)}')
print('================================================\n') 
print(classification_report(y_test, predicted, zero_division=0))  

Random Forest Accuracy: 0.9379599345870809
Random Forest Precision: 0.8143306744170957
Random Forest Recall: 0.9000612601767735
Random Forest f1_score: 0.8499015584348626
confusion matrix:
 [[ 1683   291]
 [  923 16671]]

              precision    recall  f1-score   support

    negative       0.65      0.85      0.73      1974
    positive       0.98      0.95      0.96     17594

    accuracy                           0.94     19568
   macro avg       0.81      0.90      0.85     19568
weighted avg       0.95      0.94      0.94     19568

